In [1]:
%reload_ext autoreload
%autoreload 2

import os, sys

import numpy as np
from scipy import interpolate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../matplotlibrc')

In [2]:
if os.environ['USER'] == 'yitians' and 'submit' in os.uname().nodename:
    os.environ['DH_DIR'] = '/work/submit/yitians/darkhistory/DarkHistory'
    os.environ['DM21CM_DATA_DIR'] = '/data/submit/yitians/DM21cm'
sys.path.append(os.environ['DH_DIR'])
sys.path.append('..')

In [15]:
import main

import dm21cm.physics as phys

## config

In [13]:
run_name = 'rundh_phph_mh3_21reion'

# DM turns on
#injection_modifier_func = lambda rs: float(rs < 41)
injection_modifier_func = None

# custom xe levels
base_records = np.load(f'../data/run_info/base_records.npy', allow_pickle=True).item()
def xe_reion_func(rs):
    return (1 + phys.chi) * interpolate.interp1d(1 + base_records['z'], base_records['x_e'])(rs)
def xH_reion_func(rs):
    return xe_reion_func(rs) / (1 + phys.chi)

In [17]:
base_records['z'][0]

41.56984859939324

In [18]:
reion_rs = 41
end_rs = base_records['z'][-1] + 1

soln = main.evolve(
    DM_process='swave', mDM=1e10, sigmav=1e-23, primary='phot_delta',
    start_rs=3000, end_rs=end_rs,
    coarsen_factor=12, backreaction=True, reion_switch=True,
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    verbose=1, injection_modifier_func=injection_modifier_func,
    xe_reion_func=xH_reion_func, reion_rs=reion_rs
)

Loading time: 0.209 s




  0%|          | 0/506 [00:00<?, ?it/s]

  0%|          | 1/506 [00:00<01:32,  5.47it/s]

Initialization time: 0.291 s




  0%|          | 2/506 [00:01<09:00,  1.07s/it]

  1%|          | 3/506 [00:03<10:41,  1.28s/it]

  1%|          | 4/506 [00:04<11:38,  1.39s/it]

  1%|          | 5/506 [00:06<12:30,  1.50s/it]

  1%|          | 6/506 [00:08<12:06,  1.45s/it]

  1%|▏         | 7/506 [00:09<12:23,  1.49s/it]

  2%|▏         | 8/506 [00:11<12:17,  1.48s/it]

  2%|▏         | 9/506 [00:12<13:22,  1.61s/it]

  2%|▏         | 10/506 [00:14<13:15,  1.60s/it]

  2%|▏         | 11/506 [00:16<13:40,  1.66s/it]

  2%|▏         | 12/506 [00:19<16:43,  2.03s/it]

  3%|▎         | 13/506 [00:21<16:34,  2.02s/it]

  3%|▎         | 14/506 [00:23<16:30,  2.01s/it]

  3%|▎         | 15/506 [00:25<16:57,  2.07s/it]

  3%|▎         | 16/506 [00:27<16:54,  2.07s/it]

  3%|▎         | 17/506 [00:29<16:46,  2.06s/it]

  4%|▎         | 18/506 [00:31<15:32,  1.91s/it]

  4%|▍         | 19/506 [00:32<14:36,  1.80s/it]

  4%|▍         | 20/506 [00:33<13:29,  1.66s/it]

  4%|▍         | 21/506 [00:35<13:23,  1.66s/it]

  4%|▍

KeyboardInterrupt: 

In [15]:
pickle.dump(soln, open(f'../data/{run_name}_soln.p', 'wb'))

In [16]:
arr_records = {
    'z' : soln['rs'] - 1,
    'x_e' : soln['x'][:,0],
    'T_k' : soln['Tm']/phys.kB,
    'f_heat' : soln['f']['low']['heat'] + soln['f']['high']['heat'],
    'f_ion' : soln['f']['low']['H ion']  + soln['f']['high']['H ion'] + \
              soln['f']['low']['He ion'] + soln['f']['high']['He ion'],
    'f_exc' : soln['f']['low']['exc'] + soln['f']['high']['exc'],
}

z_low, z_high = 6, 44
z_arr = arr_records['z'][::-1]
i_low = np.searchsorted(z_arr, z_low)
i_high = np.searchsorted(z_arr, z_high)

for k in arr_records.keys():
    arr_records[k] = arr_records[k][::-1][i_low:i_high]

In [17]:
np.save(f'../data/run_info/{run_name}_records', arr_records)